In [4]:
# Import necessary libraries
from langchain_community.document_loaders import WebBaseLoader  # Load web content into LangChain workflows
import bs4  # Parse and scrape HTML or XML data
from fpdf import FPDF  # Generate PDF files programmatically
import os  # Manage file system and environment operations

# Install dependencies (uncomment if required)
# !pip install fpdf langchain_community bs4

# Function to clean the content
def clean_content(content):
    """
    Cleans and formats the raw content by removing extra spaces and empty lines.
    """
    cleaned_content = []
    for line in content:
        # Remove leading/trailing spaces and condense multiple spaces
        line = " ".join(line.split())
        if line:  # Only add non-empty lines
            cleaned_content.append(line)
    return cleaned_content

# Function to save content to a PDF file
def save_to_pdf(content, folder_name, file_name):
    """
    Saves cleaned content to a PDF file.
    """
    # Create the output directory if it doesn't exist
    os.makedirs(folder_name, exist_ok=True)

    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    # Write each line to the PDF
    for line in content:
        try:
            # Handle encoding issues
            line = line.encode('latin-1', 'replace').decode('latin-1')
        except UnicodeEncodeError:
            line = "Invalid Characters Detected"
        pdf.multi_cell(0, 10, line.strip())

    # Save the PDF to the specified path
    output_path = os.path.join(folder_name, file_name)
    pdf.output(output_path)
    print(f"PDF saved at {output_path}")

# Function to scrape the data for a specific dataset
def scrape_dataset(dataset_name, urls, classes):
    """
    Scrapes content from a list of URLs and saves it as a PDF.
    """
    try:
        print(f"Scraping data for {dataset_name}...")
        all_content = []

        for url in urls:
            try:
                # Initialize WebBaseLoader with the specified URL and class filters
                loader = WebBaseLoader(
                    web_paths=[url],
                    bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=classes))
                )
                documents = loader.load()
                raw_content = [doc.page_content for doc in documents]

                if not raw_content:
                    print(f"No content found for URL: {url}. Skipping.")
                    continue

                all_content.extend(raw_content)
            except Exception as url_error:
                print(f"Error scraping URL {url}: {url_error}. Skipping.")

        if not all_content:
            print(f"No valid content found for {dataset_name}. Skipping PDF generation.")
            return

        # Clean the scraped content
        cleaned_content = clean_content(all_content)

        # Save content to a PDF in the dataset's folder
        save_to_pdf(cleaned_content, "rgukt_datasets_21", f"{dataset_name}.pdf")
    except Exception as e:
        print(f"Error scraping {dataset_name}: {e}")

# Main function to manage all datasets
def scrape_all_datasets():
    """
    Manages multiple datasets and initiates scraping for each.
    """
    datasets = {
        "about_rgukt": {
            "urls": [
                "http://www.rgukt.ac.in/about-introduction.html",
                "http://www.rgukt.ac.in/vision-mission.html",
                "http://www.rgukt.ac.in/stu-campuslife.html",
                "http://www.rgukt.ac.in/anti-ragging.html",
                "https://www.rgukt.ac.in/about-introduction.html",
                "https://www.rgukt.ac.in/vision-mission.html",
                "https://www.rgukt.ac.in/vc.html",
                "https://www.rgukt.ac.in/gc.html",
                "https://www.rgukt.ac.in/administration-section.html",
                "https://www.rgukt.ac.in/cd.html",
                "https://www.rgukt.ac.in/academicprogrammes.html",
                "https://www.rgukt.ac.in/curricula.html",
                "https://www.rgukt.ac.in/academiccalender.html",
                "https://www.rgukt.ac.in/departments.html",
                "https://www.rgukt.ac.in/examination.html",
                "https://www.rgukt.ac.in/index.html",
                "https://www.rgukt.ac.in/cse.html",
                "https://www.rgukt.ac.in/che.html",
                "https://www.rgukt.ac.in/ce.html",
                "https://www.rgukt.ac.in/ece.html",
                "https://www.rgukt.ac.in/mme.html",
                "https://www.rgukt.ac.in/me.html",
                "https://www.rgukt.ac.in/hostels.html",
                "https://www.rgukt.ac.in/library/",
                "https://www.rgukt.ac.in/hospital.html",
                "https://www.rgukt.ac.in/placement/",
                'https://www.rgukt.ac.in/alumni.html',
                "https://www.rgukt.ac.in/contactus.html",
                "http://www.rgukt.ac.in/cse-faculty.html"
            ],
            "classes": ["page-row"]  # HTML class names to target specific content
        }
    }

    # Scrape each dataset
    for dataset_name, details in datasets.items():
        scrape_dataset(dataset_name, details["urls"], details["classes"])

# Entry point for the script
if __name__ == "__main__":
    scrape_all_datasets()


Scraping data for about_rgukt...
PDF saved at rgukt_datasets_21/about_rgukt.pdf
